In [243]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn 
import matplotlib as mpl
import matplotlib.pyplot as plt      
import math
from collections import Counter
%matplotlib inline                         
from sklearn import preprocessing, linear_model, model_selection,tree
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from sklearn.metrics import confusion_matrix , accuracy_score , f1_score, classification_report, make_scorer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier
from IPython.display import Image, display  
import pydotplus 
from scipy import misc
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

In [244]:
def load_csv(file_name):
    df=pd.read_csv(file_name)
    return df

In [245]:
def binary_raiting_base_on_success(df):
    df_copy=df.copy()
    df_copy["raiting"]=np.where(df_copy["raiting"] >= 4.5 ,1,0)
    return df_copy

In [246]:
def transfer_to_categorial(df):
    df_copy= df.copy()
    df_copy = pd.get_dummies(df,columns=['category'],prefix=["category"])
    
    del df_copy['game_name']
    del df_copy['seller']
    
    
    return df_copy

In [247]:
def grid_search_cv(df,XTrain,yTrain):

    parameters = {'max_depth':[2,4,6],"min_samples_split":[5,10,20,25,50] }
    dt = RandomForestClassifier()

    clf = GridSearchCV(dt, parameters,scoring=make_scorer(metrics.f1_score, greater_is_better=True))
    clf.fit(XTrain, yTrain)

    print("best parameter set is:",clf.best_params_," and its score was",clf.best_score_)
    
    return clf.best_params_,clf.best_score_

In [248]:
def train_test_split_function(X,y):
    X_train,X_test,y_train ,y_test = train_test_split(X,y,test_size = 0.3,random_state= 0)
    return X_train, X_test, y_train, y_test
    

In [249]:
def normalize_values(X_train , X_test):
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test

In [250]:
def train_model(X_train, y_train,X_test,y_test):
    forest =  RandomForestClassifier(bootstrap=True, n_estimators=300, random_state=0)

    trained_forest = forest.fit(X_train, y_train) 

    y_pred_train = trained_forest.predict(X_train)
    print('f1_score on training data= ', metrics.f1_score(y_true = y_train, y_pred = y_pred_train))

    y_pred = trained_forest.predict(X_test)
    print('f1_score on test data= ', metrics.f1_score(y_true = y_test, y_pred = y_pred))
    return y_pred , trained_forest

In [251]:
def confusion_matrix_eval(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    print("Accuracy: " ,f1_score(average = 'weighted' , y_true= y_test ,y_pred = y_pred))
    cm

In [252]:
def split_to_x_and_y(df):
    df_copy=df.copy()
    y=df_copy["raiting"]
    del df_copy["raiting"]
    X= df_copy
    return X,y

In [253]:
file_name = "C:/Users/tomer/Desktop/תואר ראשון/שנה ב/file_games_EDA.csv"
df = load_csv(file_name)
del df["Unnamed: 0"] 
df = binary_raiting_base_on_success(df) #1 for 4.5 rating and above, 0 for 4.4 and under

df_numeric = transfer_to_categorial(df) #transfering string to categorical


X,y = split_to_x_and_y(df_numeric) #creating X matrix and y column

X_train, X_test, y_train, y_test = train_test_split_function(X,y) #spliting to x train test and y train test

X_train , X_test = normalize_values(X_train , X_test) #normalizing values between 0 to 1

y_pred, model = train_model(X_train, y_train,X_test,y_test) #first model training


best_param, best_score = grid_search_cv(df_numeric
,X_train,y_train) #finding the best scores of the last model
best_model = RandomForestClassifier(bootstrap=True, random_state = 0 , n_estimators=300, max_depth = best_param.get("max_depth") , min_samples_split = best_param.get("min_samples_split") )
#training the model due to the best params from the last model we trained
best_model.fit(X_train , y_train)
#training the model again
y_pred = best_model.predict(X_test)
#predicting y according the our best model

print('Accuracy after improvments = ', metrics.f1_score(y_true = y_test, y_pred = y_pred))
print(classification_report(y_test, y_pred))

metrics.confusion_matrix(y_test , y_pred) 
#the values on the diagonal are values that are correct,
# and other diagonal is the one the model failed to predict





f1_score on training data=  1.0
f1_score on test data=  0.7857142857142857
best parameter set is: {'max_depth': 6, 'min_samples_split': 20}  and its score was 0.7957302645024992
Accuracy after improvments =  0.7907995618838992
              precision    recall  f1-score   support

           0       0.63      0.50      0.56       241
           1       0.75      0.84      0.79       432

    accuracy                           0.72       673
   macro avg       0.69      0.67      0.67       673
weighted avg       0.71      0.72      0.71       673



array([[121, 120],
       [ 71, 361]], dtype=int64)